## Import

In [3]:
import random
import os
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import warnings
warnings.filterwarnings('ignore')

In [4]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda', index=0)

## Hyperparameter Setting

In [5]:
CFG = {
    'TRAIN_WINDOW_SIZE':90, # 90일치로 학습
    'PREDICT_SIZE':21, # 21일치 예측
    'EPOCHS':10,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':4096,
    'SEED':41,
    'STRIDE':1
}

In [6]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## 데이터 불러오기

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
train_data = pd.read_csv('./open/train.csv').drop(columns=['ID', '제품'])

In [8]:
brand = pd.read_csv('./open/brand_keyword_cnt.csv')

In [9]:
sales = pd.read_csv('./open/sales.csv')

In [10]:
empty_brand = brand.set_index('브랜드').T.isnull().sum().sort_values(ascending=False)[:35].index

In [11]:
periods = pd.to_datetime(train_data.columns[4:])
weekday = periods.weekday

In [12]:
empty_brand = brand.set_index('브랜드').T.isnull().sum().sort_values(ascending=False)[:35].index
col = brand.columns[1:]
mean = brand.iloc[:,1:].mean()
for b in empty_brand:
    brand.loc[brand['브랜드']==b,col] = mean.values
    
# brand = brand.fillna(0)

In [13]:
cost = sales.iloc[:6:] / train_data.iloc[:,4:]
cost.fillna(method='ffill',inplace=True)
cost.fillna(method='bfill',inplace=True)

### 데이터 전처리

In [14]:
# 숫자형 변수들의 min-max scaling을 수행하는 코드입니다.
numeric_cols = train_data.columns[4:]
# 칵 column의 min 및 max 계산
min_values = train_data[numeric_cols].min(axis=1)
max_values = train_data[numeric_cols].max(axis=1)
# 각 행의 범위(max-min)를 계산하고, 범위가 0인 경우 1로 대체
ranges = max_values - min_values
ranges[ranges == 0] = 1  ##기존 코드에서는 0으로 처리함
# min-max scaling 수행
train_data[numeric_cols] = (train_data[numeric_cols].subtract(min_values, axis=0)).div(ranges, axis=0)
# max와 min 값을 dictionary 형태로 저장
scale_min_dict = min_values.to_dict()
scale_max_dict = max_values.to_dict()

In [15]:
# Label Encoding
label_encoder = LabelEncoder()
categorical_columns = ['대분류', '중분류', '소분류']

for col in categorical_columns:
    label_encoder.fit(train_data[col])
    train_data[col] = label_encoder.transform(train_data[col])

label_encoder.fit(train_data['브랜드'])
train_data['브랜드'] = label_encoder.transform(train_data['브랜드'])
brand['브랜드'] = label_encoder.transform(brand['브랜드'])

## make dataset

In [19]:
def make_train_data(data, train_size=CFG['TRAIN_WINDOW_SIZE'], predict_size=CFG['PREDICT_SIZE'],stride=CFG['STRIDE']):
    '''
    학습 기간 블럭, 예측 기간 블럭의 세트로 데이터를 생성
    data : 일별 판매량
    train_size : 학습에 활용할 기간
    predict_size : 추론할 기간
    '''
    num_rows = len(data) #train의 행 => 제품개수
    window_size = train_size + predict_size

#     input_data = np.empty((num_rows * (len(data.columns) - window_size + 1) //2, train_size, len(data.iloc[0, :4]) + 2))
#     target_data = np.empty((num_rows * (len(data.columns) - window_size + 1)//2, predict_size))

    input_data = np.empty((num_rows * len(range(0,len(data.columns) - 4 - window_size + 1,stride)), train_size, len(data.iloc[0, :4]) + 3))
    target_data = np.empty((num_rows * len(range(0,len(data.columns) - 4 - window_size + 1,stride)), predict_size))

    for i in tqdm(range(num_rows)): #각 ID 마다
        encode_info = np.array(data.iloc[i, :4]) #분류 정보
        sales_data = np.array(data.iloc[i, 4:]) #판매 정보
        temp_brand = brand[brand['브랜드']==encode_info[-1]].values[0][1:]

        assert len(temp_brand)==len(sales_data)

        for idx,j in enumerate(range(0,len(sales_data) - window_size + 1, stride)):
            brand_window = temp_brand[j : j + window_size]
            sales_window = sales_data[j : j + window_size]
            weekday_window = weekday.values[j : j + window_size]
            temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)),
                                         brand_window[:train_size],
                                         weekday_window[:train_size],
                                         sales_window[:train_size]))
            # (분류정보 + 브랜드 4열이 90번 아래로 주르륵) + (90일치 학습데이터 열 추가)
            input_data[i * len(range(0,len(data.columns) - 4 - window_size + 1,stride)) + idx] = temp_data
            target_data[i * len(range(0,len(data.columns) - 4 - window_size + 1,stride)) + idx] = sales_window[train_size:]

    return input_data, target_data

In [20]:
def make_predict_data(data, train_size=CFG['TRAIN_WINDOW_SIZE']):
    '''
    평가 데이터(Test Dataset)를 추론하기 위한 Input 데이터를 생성
    data : 일별 판매량
    train_size : 추론을 위해 필요한 일별 판매량 기간 (= 학습에 활용할 기간)
    '''
    num_rows = len(data)
    
    input_data = np.empty((num_rows, train_size, len(data.iloc[0, :4]) + 3))

    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :4])
        sales_data = np.array(data.iloc[i, -train_size:])
        temp_brand = brand[brand['브랜드']==encode_info[-1]].values[0][-train_size:]

        sales_window = sales_data[-train_size : ]
        brand_window = temp_brand[-train_size : ]
        weekday_window = weekday.values[-train_size : ]
        temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)),
                                     brand_window[:train_size],
                                     weekday_window[:train_size],
                                     sales_window[:train_size]))
        input_data[i] = temp_data

    return input_data

In [21]:
def make_psfa_data(data, train_size=CFG['TRAIN_WINDOW_SIZE'], predict_size=CFG['PREDICT_SIZE']):
    
    num_rows = len(data)
    window_size = train_size + predict_size
    
    input_data = np.empty((num_rows, train_size, len(data.iloc[0, :4]) + 3))
    target_data = np.empty((num_rows, predict_size))

    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :4])
        sales_data = np.array(data.iloc[i, -window_size:])
        temp_brand = brand[brand['브랜드']==encode_info[-1]].values[0][-window_size:]
        temp_weekday = weekday.values[-window_size:]

        temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)),
                                     temp_brand[:train_size],
                                     temp_weekday[:train_size],
                                     sales_data[:train_size]))
        input_data[i] = temp_data
        target_data[i] = sales_data[train_size:]

    return input_data, target_data

In [22]:
train_input, train_target = make_train_data(train_data)

100%|██████████| 15890/15890 [01:58<00:00, 134.57it/s]


In [ ]:
item_info = np.array([int(i[0,0]) for i in train_input])

In [ ]:
len(item_info)

1859130

In [23]:
test_input = make_predict_data(train_data)

100%|██████████| 15890/15890 [00:23<00:00, 675.20it/s]


In [24]:
psfa_input, psfa_target = make_psfa_data(train_data)

100%|██████████| 15890/15890 [00:23<00:00, 676.65it/s]


## train_test_split

#### 그냥 0.8 / 0.2 split

In [ ]:
# # Train / Validation Split
# data_len = len(train_input)
# val_input = train_input[-int(data_len*0.2):]
# val_target = train_target[-int(data_len*0.2):]
# train_input = train_input[:-int(data_len*0.2)]
# train_target = train_target[:-int(data_len*0.2)]

# val_item = item_info[-int(data_len*0.2):]
# train_item = item_info[:-int(data_len*0.2)]

#### item 별로 0.8 / 0.2 split (웬만하면 이걸로)

In [ ]:
per_item = int(len(train_input)/15890)
t_len = int(per_item * 0.8)
v_len = per_item - t_len

t_list = []
v_list = []

for i in range(0,len(train_input),per_item):
    t_list.extend(list(range(i,i+t_len)))
    v_list.extend(list(range(i+t_len,i+per_item)))

val_input = train_input[v_list]
val_target = train_target[v_list]
train_input = train_input[t_list]
train_target = train_target[t_list]

val_item = item_info[v_list]
train_item = item_info[t_list]

#### random train_test_split

In [ ]:
# from sklearn.model_selection import train_test_split
# train_input,val_input,train_target,val_target,train_item,val_item = train_test_split(train_input,
#                                                                                      train_target,
#                                                                                      item_info,
#                                                                                      shuffle=True,
#                                                                                      random_state=42,
#                                                                                      test_size=0.2)
# ## stratify = item_info 할까 말까

In [ ]:
train_input.shape, train_target.shape, val_input.shape, val_target.shape, test_input.shape, psfa_input.shape, psfa_target.shape

((1477770, 90, 6),
 (1477770, 21),
 (381360, 90, 6),
 (381360, 21),
 (15890, 90, 6),
 (15890, 90, 6),
 (15890, 21))

## PSFA

In [ ]:
indexs_bigcat={}
for bigcat in train_data['대분류'].unique():
    indexs_bigcat[bigcat] = list(train_data.loc[train_data['대분류']==bigcat].index)

indexs_bigcat.keys()

## 대분류별로 15890개 item의 index

dict_keys([1, 2, 0, 4, 3])

In [ ]:
##item별로 딱 한번만 예측해서 15890개만 존재할때 사용

def PSFA(pred, target):
    PSFA = 1
    for cat in range(5):
        ids = indexs_bigcat[cat]
        for day in range(CFG['PREDICT_SIZE']):
            total_sell = np.sum(target[ids, day]) # day별 총 판매량
            pred_values = pred[ids, day] # day별 예측 판매량
            target_values = target[ids, day] # day별 실제 판매량

            # 실제 판매와 예측 판매가 같은 경우 오차가 없는 것으로 간주
            denominator = np.maximum(target_values, pred_values)
            diffs = np.where(denominator!=0, np.abs(target_values - pred_values) / denominator, 0)

            if total_sell != 0:
                sell_weights = target_values / total_sell  # Item별 day 총 판매량 내 비중
            else:
                sell_weights = np.ones_like(target_values) / len(ids)  # 1 / len(ids)로 대체

            if not np.isnan(diffs).any():  # diffs에 NaN이 없는 경우에만 PSFA 값 업데이트
                PSFA -= np.sum(diffs * sell_weights) / (CFG['PREDICT_SIZE'] * 5)


    return PSFA

### Custom Dataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __getitem__(self, index):
        if self.Y is not None:
            return torch.Tensor(self.X[index]), torch.Tensor(self.Y[index])
        return torch.Tensor(self.X[index])

    def __len__(self):
        return len(self.X)

In [ ]:
train_dataset = CustomDataset(train_input, train_target)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_input, val_target)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

### 모델 선언

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, input_size=6, hidden_size=512, num_layers=1, output_size=CFG['PREDICT_SIZE']):
        super(BaseModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, hidden_size//2),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(hidden_size//2, output_size)
        )

        self.actv = nn.ReLU()

    def forward(self, x):
        # x shape: (B, TRAIN_WINDOW_SIZE, 5)
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size, x.device)

        # LSTM layer
        lstm_out, hidden = self.lstm(x, hidden)

        # Only use the last output sequence
        last_output = lstm_out[:, -1, :]

        # Fully connected layer
        output = self.actv(self.fc(last_output))

        return output.squeeze(1)

    def init_hidden(self, batch_size, device):
        # Initialize hidden state and cell state
        return (torch.zeros(self.num_layers, batch_size, self.hidden_size, device=device),
                torch.zeros(self.num_layers, batch_size, self.hidden_size, device=device))

### 모델 학습

In [ ]:
def train(model, optimizer, train_loader, val_loader, device):
    model.to(device)
    criterion = nn.MSELoss().to(device)
    best_loss = 9999999
    best_psfa = 0
    best_model = None

    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        train_mae = []
        for X, Y in tqdm(iter(train_loader)):
            X = X.to(device)
            Y = Y.to(device)

            optimizer.zero_grad()

            output = model(X)
            loss = criterion(output, Y)

            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

        #val_loss = validation(model, val_loader, criterion, device)
        val_psfa, val_loss = validation(model, val_loader, criterion, device)
        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss):.5f}] Val Loss : [{val_loss:.5f}] Val_PSFA : [{val_psfa:.3f}]')

#         if best_loss > val_loss:
#             best_loss = val_loss
#             best_model = model
#             print('Model Saved')

        if best_psfa < val_psfa:
            best_psfa = val_psfa
            best_model = model
            patient = 0
            print('Model Saved')
        else:
            if epoch>50:
                patient+=1
        if patient>30:
            print('Early Stopping patient over 30 !')
            break
    return best_model

In [ ]:
def validation(model, val_loader, criterion, device):
    pred = []
    target = []
    val_loss = []

    with torch.no_grad():
        for X, Y in iter(val_loader):
            X = X.to(device)
            Y = Y.to(device)

            output = model(X)
            loss = criterion(output, Y)
            val_loss.append(loss.item())
            output = output.cpu().numpy()
            pred.extend(output)

            Y = Y.cpu().numpy()
            target.extend(Y)


    pred = np.array(pred)
    target = np.array(target)

    # 추론 결과를 inverse scaling
    for idx in range(len(pred)): #15889
        pred[idx, :] = pred[idx, :] * (scale_max_dict[val_item[idx]] - scale_min_dict[val_item[idx]]) + scale_min_dict[val_item[idx]]
        target[idx, :] = target[idx, :] * (scale_max_dict[val_item[idx]] - scale_min_dict[val_item[idx]]) + scale_min_dict[val_item[idx]]

    # 결과 후처리
    ##### 후처리 해야하나 싶기도하고? #####
    pred = np.round(pred, 0).astype(int)
    target = np.round(target, 0).astype(int)

    return PSFA(pred, target),np.mean(val_loss)

## Run !!

In [ ]:
model = BaseModel()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
infer_model = train(model, optimizer, train_loader, val_loader, device)

## 모델 추론

In [ ]:
test_dataset = CustomDataset(test_input, None)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader, device):
    predictions = []

    with torch.no_grad():
        for X in tqdm(iter(test_loader)):
            X = X.to(device)

            output = model(X)

            # 모델 출력인 output을 CPU로 이동하고 numpy 배열로 변환
            output = output.cpu().numpy()

            predictions.extend(output)

    return np.array(predictions)

In [ ]:
pred = inference(infer_model, test_loader, device)

  0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# 추론 결과를 inverse scaling
for idx in range(len(pred)):
    pred[idx, :] = pred[idx, :] * (scale_max_dict[idx] - scale_min_dict[idx]) + scale_min_dict[idx]

# 결과 후처리
pred = np.round(pred, 0).astype(int)

## Submission

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/LG_AIMERS/data/sample_submission.csv')
submit.head()

In [ ]:
submit.iloc[:,1:] = pred
submit.head()

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
submit.to_csv('/content/drive/MyDrive/LG_AIMERS/submission/dropoutlier_submission.csv', index=False)